## Mohamed Ben ALI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time

In [ ]:
C = 1 # constante à déterminer
Q = 1e-2 # covariance du bruit d'état
R = 10 # covariance du bruit de mesure

In [ ]:
z = norm.rvs(loc = C, scale = np.sqrt(R), size = 500) # prend 500 valeurs aléatoires centrées en C et de variance R
t = np.arange(0, len(z), 1)
plt.plot(t, z, 'bo')
plt.title('Les mesures')
plt.xlabel('Temps (s)')
plt.ylabel('Les valeurs de la mesures')
plt.grid()
plt.show()

In [ ]:
xh = np.zeros(shape = len(z)) # vecteur des estimations
x0 = norm.rvs(loc = 0, scale = np.sqrt(2), size = 1) # condition initiale sur x
P = np.zeros(shape = len(z)) # vecteur des variances des erreurs d'estimation
P0 = 1 # condition initiale sur P

xh[0] = x0
P[0] = P0 + Q # on ajoute le bruit d'état

In [ ]:
# Algorithme de Kalman
for p in range(0, len(z), 1):
    K = P[p] / (P[p] + R) # calcul du gain de Kalman
    xh[p] = xh[p] + K * (z[p] - xh[p]) # estimation
    P[p] = P[p] - K * P[p] # variance de l'erreur d'estimation
    if p < len(z) - 1: # passage à l'itération suivante
        xh[p + 1] = xh[p]
        P[p + 1] = P[p] + Q

In [ ]:
un = np.ones(shape = len(z))
plt.plot(t, z, 'go', label = 'Les mesures')
plt.plot(t, xh, 'r+', label = 'L\'estimation')
plt.plot(t, un, 'b', label = 'La valeur de la constante')
plt.title('Estimation de la constante C')
plt.xlabel('Temps (s)')
plt.ylabel('Les valeurs l\'etimation')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(t, P)
plt.title('evolution de la matrice de covariance')
plt.xlabel('Temps (s)')
plt.ylabel('Valeurs de la matrice de covariance')
plt.grid()
plt.show()

In [ ]:
plt.plot(t, un - xh, 'ys')
plt.title('Erreur d\'estimation')
plt.xlabel('Temps (s)')
plt.ylabel('Valeurs de l\'erreur')
plt.grid()
plt.show()

## Test de différentes valeurs de Q

In [ ]:
def kalman(Q) :
    xh = np.zeros(shape = len(z)) # vecteur des estimations
    x0 = norm.rvs(loc = 0, scale = np.sqrt(2), size = 1) # condition initiale sur x
    P = np.zeros(shape = len(z)) # vecteur des variances des erreurs d'estimation
    P0 = 1 # condition initiale sur P

    xh[0] = x0
    P[0] = P0 + Q # on ajoute le bruit d'état
    
    # Algorithme de Kalman
    for p in range(0, len(z), 1):
        K = P[p] / (P[p] + R) # calcul du gain de Kalman
        xh[p] = xh[p] + K * (z[p] - xh[p]) # estimation
        P[p] = P[p] - K * P[p] # variance de l'erreur d'estimation
        if p < len(z) - 1: # passage à l'itération suivante
            xh[p + 1] = xh[p]
            P[p + 1] = P[p] + Q
     
     
    plt.figure(figsize=(30,10))       
    plt.subplot(131)
    un = np.ones(shape = len(z))
    plt.plot(t, z, 'go', label = 'Les mesures')
    plt.plot(t, xh, 'r+', label = 'L\'estimation')
    plt.plot(t, un, 'b', label = 'La valeur de la constante')
    plt.title(f'Estimation de la constante C pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Les valeurs l\'etimation')
    plt.legend()
    plt.grid()
    
    plt.subplot(132)
    plt.plot(t, P)
    plt.title(f'Evolution de la matrice de covariance pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Valeurs de la matrice de covariance')
    plt.grid()
    
    plt.subplot(133)
    plt.plot(t, un - xh, 'ys')
    plt.title(f'Erreur d\'estimation pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Valeurs de l\'erreur')
    plt.grid()
    plt.show()
        

In [ ]:
Q_list = [10**(k) for k in range(-15,5)]
Q_list.insert(0,0)

for Q in Q_list :
    print(f"Estimation for Q = {Q}")
    kalman(Q)

## Commentaire : effet de la valeur de Q

* On observe que quand Q est "petit", le filtre de Kalman a un temps de réponse important mais il n'oscille pas beaucoup autour de la valeur limite : on a donc une erreur résiduelle assez faible (de l'ordre de $10^{-1}$).
* A l'inverse, quand Q est "grand", le filtre de Kalman a un temps de réponse faible (i.e. il se rapproche de la constante à estimer assez rapidement), mais il oscille beaucoup autour de la valeur limite : on a donc une erreur résiduelle importante et variable.
* On peut s'intéresser au cas limite "Q tend vers $+\infty$". On observe que lorsque Q est très grand, l'estimation devient la mesure elle-même. Cela s'explique par le fait que quand Q tend vers $+\infty$, le gain de Kalman tend vers 1 et à chaque itération on se contente de donner à l'estimation la valeur de la mesure (`xh[p] = xh[p] + K * (z[p] - xh[p])` dans l'algorithme de Kalman).

En conclusion, la valeur de Q, le bruit d'état, représente les phénomènes que l'on n'a pas pris en compte dans notre modèle. Sa valeur dépend donc de la "confiance" que l'on a dans notre modélisation. Ainsi, si on prend Q très grand, cela signifie que l'on a peu confiance dans notre modèle et on se base uniquement sur les mesures (ce qui explique le cas extrême "Q tend vers $+\infty$"). 

De façon générale, le choix de Q influence le comportement dynamique du filtre de Kalman (rapidité de convergence et précision).

## Estimation d'un signal carré

In [ ]:
C = 1 # constante à déterminer
Q = 1e-2 # covariance du bruit d'état
R = 0.1 # covariance du bruit de mesure

# Signal carré à estimer
x = np.zeros(shape = 100)
for _ in range(0,3):
    x_up = np.ones(shape = 100)
    x_down = np.zeros(shape = 100)
    x = np.hstack((x,x_up,x_down)) 

# Génération de points aléatoires
z = norm.rvs(loc = 0, scale = np.sqrt(R), size = 100) 
for _ in range(0,3):
    z1 = norm.rvs(loc = C, scale = np.sqrt(R), size = 100) 
    z0 = norm.rvs(loc = 0, scale = np.sqrt(R), size = 100)
    z = np.hstack((z,z1,z0)) 
    
 
t = np.arange(0, len(z), 1)
plt.plot(t, z, 'bo')
plt.plot(t, x, 'r')
plt.title('Les mesures et le signal carré')
plt.xlabel('Temps (s)')
plt.ylabel('Les valeurs de la mesures')
plt.grid()
plt.show()

In [ ]:
def kalman_square(Q):
    xh = np.zeros(shape = len(z)) # vecteur des estimations
    x0 = norm.rvs(loc = 0, scale = np.sqrt(2), size = 1) # condition initiale sur x
    P = np.zeros(shape = len(z)) # vecteur des variances des erreurs d'estimation
    P0 = 1 # condition initiale sur P

    xh[0] = x0
    P[0] = P0 + Q # on ajoute le bruit d'état
    
    # Algorithme de Kalman
    for p in range(0, len(z), 1):
        K = P[p] / (P[p] + R) # calcul du gain de Kalman
        xh[p] = xh[p] + K * (z[p] - xh[p]) # estimation
        P[p] = P[p] - K * P[p] # variance de l'erreur d'estimation
        if p < len(z) - 1: # passage à l'itération suivante
            xh[p + 1] = xh[p]
            P[p + 1] = P[p] + Q
     
     
    plt.figure(figsize=(30,10))       
    plt.subplot(131)
    un = np.ones(shape = len(z))
    plt.plot(t, z, 'go', label = 'Les mesures')
    plt.plot(t, xh, 'r+', label = 'L\'estimation')
    plt.plot(t, x, 'b', label = 'La valeur de la constante')
    plt.title(f'Estimation de la constante C pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Les valeurs l\'etimation')
    plt.legend()
    plt.grid()
    
    plt.subplot(132)
    plt.plot(t, P)
    plt.title(f'Evolution de la matrice de covariance pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Valeurs de la matrice de covariance')
    plt.grid()
    
    plt.subplot(133)
    plt.plot(t, un - xh, 'ys')
    plt.title(f'Erreur d\'estimation pour Q = {Q}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Valeurs de l\'erreur')
    plt.grid()
    plt.show()
    

In [ ]:
Q_list = [10**(k) for k in range(-15,5)]
Q_list.insert(0,0)

for Q in Q_list :
    print(f"Estimation for Q = {Q}")
    kalman_square(Q)

## Commentaire : estimation d'un signal carré

Encore une fois, on essaie différentes valeurs de Q pour estimer le signal carré x. On peut noter que l'on a réduit le bruit de mesure R car la valeur de 10 était beaucoup trop grande pour un signal variant entre 0 et 1 (il serait noyé dans le bruit).

On fait alors les observations suivantes :

* Tout ce qui a été dit précédemment sur l'influence de Q est encore valable.
* Ce qui est remarquable ici est que quand Q est petit, le temps de réponse est trop faible pour que l'estimation puisse s'approcher du signal sur chacun des intervalles ("le filtre n'a pas le temps d'estimer la constante 1 avant qu'elle devienne 0 et inversement").
* Pour Q grand, le temps de réponse est suffisant pour se rapprocher du signal carré réel mais les oscillations deviennent alors très importantes.

## Peut-on mieux estimer le signal carré si R est "petit" ?

In [ ]:
C = 1 # constante à déterminer
Q = 1e-2 # covariance du bruit d'état
R = 0.001 # covariance du bruit de mesure --> on la diminue !

# Signal carré à estimer
x = np.zeros(shape = 100)
for _ in range(0,3):
    x_up = np.ones(shape = 100)
    x_down = np.zeros(shape = 100)
    x = np.hstack((x,x_up,x_down)) 

# Génération de variable aléatoires
z = norm.rvs(loc = 0, scale = np.sqrt(R), size = 100) 
for _ in range(0,3):
    z1 = norm.rvs(loc = C, scale = np.sqrt(R), size = 100) 
    z0 = norm.rvs(loc = 0, scale = np.sqrt(R), size = 100)
    z = np.hstack((z,z1,z0)) 
    
 
t = np.arange(0, len(z), 1)
plt.plot(t, z, 'bo')
plt.plot(t, x, 'r')
plt.title('Les mesures et le signal carré')
plt.xlabel('Temps (s)')
plt.ylabel('Les valeurs de la mesures')
plt.grid()
plt.show()

Q_list = [10**(k) for k in range(-15,5)]
Q_list.insert(0,0)

for Q in Q_list :
    print(f"Estimation for Q = {Q}")
    kalman_square(Q)

## Commentaire

On remarque que cette fois, pour un bon choix de Q (e.g. Q = 1e-05), on arrive à estimer le signal de façon satisfaisante. En revanche, si Q est petit on n'arrive toujours pas à estimer le signal puisque l'on a encore le problème du temps de réponse trop faible du filtre.